In [1]:
import pandas as pd

In [2]:
path = '/content/drive/MyDrive/Aviator/Data1/data(1).csv'
data = pd.read_csv(path)
print(data)

        Unnamed: 0   Odd  Odd_Categoria  odd_entrada  \
0                0  1.15              3         11.0   
1                1  1.19              3          3.0   
2                2  5.92             10          3.0   
3                3  1.11              2         10.0   
4                4  1.06              2          2.0   
...            ...   ...            ...          ...   
999984      999984  1.71              6         10.0   
999985      999985  5.16             10          6.0   
999986      999986  1.23              3         10.0   
999987      999987  9.00             10          3.0   
999988      999988  2.26              7         10.0   

        odd_categoria_media_movel5  odd_categoria_media_movel10  \
0                              6.4                          6.5   
1                              5.8                          6.1   
2                              5.0                          5.7   
3                              6.4                         

In [3]:
data = data[['Odd_Categoria','odd_entrada',
       'odd_categoria_media_movel5', 'odd_categoria_media_movel10',
       'odd_categoria_media_movel20', 'odd_categoria_media_movel40',
       'odd_categoria_media_movel80', 'odd_categoria_media_movel160',
       'odd_categoria_media_movel320', 'odd_categoria_media_movel640',
       'odd_mediana20', 'odd_min10', 'odd_min20', 'odd_min40', 'odd_max10',
       'tendencia_media_movel10', 'categoria_frequencia20',
       'entropia_rolling5', 'entropia_rolling320',
       'correlacao_odd_categoria_media_movel10',
       'correlacao_odd_categoria_media_movel640', 'categoria_contagem',
       'variancia_rolling10', 'variancia_rolling20', 'variancia_rolling320',
       '1entrada-anterior', '2entrada-anterior', '3entrada-anterior',
       '4entrada-anterior', '5entrada-anterior', '6entrada-anterior',
       '7entrada-anterior', '8entrada-anterior', '9entrada-anterior',
       '10entrada-anterior']]

In [4]:
data2 = data

In [5]:
data = data.tail(100000).reset_index(drop=True)
data

,Odd_Categoria,odd_entrada,odd_categoria_media_movel5,odd_categoria_media_movel10,odd_categoria_media_movel20,odd_categoria_media_movel40,odd_categoria_media_movel80,odd_categoria_media_movel160,odd_categoria_media_movel320,odd_categoria_media_movel640,...,1entrada-anterior,2entrada-anterior,3entrada-anterior,4entrada-anterior,5entrada-anterior,6entrada-anterior,7entrada-anterior,8entrada-anterior,9entrada-anterior,10entrada-anterior
0,10,6.0,6.8,6.5,6.40,6.175,6.2500,6.06875,6.156250,6.050000,...,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0
1,5,10.0,7.0,7.1,6.45,6.150,6.2625,6.06250,6.165625,6.060937,...,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0
2,7,5.0,6.8,7.3,6.30,6.000,6.2250,6.03750,6.159375,6.051563,...,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0,5.0
3,2,7.0,6.4,7.0,6.55,6.050,6.2125,6.04375,6.175000,6.051563,...,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0,7.0
4,2,2.0,6.0,6.1,6.40,6.075,6.1500,5.98750,6.165625,6.051563,...,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,6,10.0,6.6,6.4,6.70,6.025,5.8625,6.06875,5.962500,6.145313,...,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0
99996,10,6.0,6.2,6.1,6.45,6.000,5.9125,6.03750,5.962500,6.146875,...,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0,6.0
99997,3,10.0,7.4,6.6,6.75,6.075,5.9375,6.03750,5.975000,6.145313,...,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0
99998,10,3.0,6.8,5.9,6.70,6.100,5.9625,6.01875,5.968750,6.139062,...,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,3.0


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from collections import deque
import random

class DQNAgent:
    def __init__(self, n_states, n_actions, learning_rate, gamma, epsilon, buffer_size=10000):
        self.n_states = n_states
        self.n_actions = n_actions
        self.learning_rate = learning_rate
        self.gamma = gamma
        self.epsilon = epsilon
        self.memory = deque(maxlen=buffer_size)

        self.model = self.build_model()
        self.model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate), loss='sparse_categorical_crossentropy')

    def build_model(self):
        model = tf.keras.Sequential([
            tf.keras.layers.Input(shape=(self.n_states,)),
            tf.keras.layers.Dense(4, activation='relu'),
            tf.keras.layers.Dense(4, activation='relu'),
            tf.keras.layers.Dense(self.n_actions, activation='softmax')
        ])
        return model

    def get_action(self, state):
        if np.random.random() < self.epsilon:
            return np.random.choice(self.n_actions)
        else:
            q_values = self.model.predict(state.reshape(1, -1), batch_size=1)
            return np.argmax(q_values)

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def replay(self, batch_size):
        if len(self.memory) < batch_size:
            return
        minibatch = random.sample(self.memory, batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target = (reward + self.gamma * np.amax(self.model.predict(next_state.reshape(1, -1), batch_size=1)))
            target_f = self.model.predict(state.reshape(1, -1), batch_size=1)
            target_f[0][action] = target
            self.model.fit(state.reshape(1, -1), np.array([action]), epochs=1, verbose=0, batch_size=batch_size)

    def update_epsilon(self, decay_rate, min_epsilon):
        if self.epsilon > min_epsilon:
            self.epsilon *= decay_rate

def calculate_reward(action, true_action):
  base_reward = 10
  good_error_reward = 5
  bad_error_penalty = -5

  extra_good_reward = 3
  extra_bad_penalty = -3
  valueable_categorias = [7,8,9,10,11]

  if action == true_action:
    return base_reward  # Maior recompensa para acertos exatos
  elif action < true_action:
    reward = good_error_reward  # Recompensa para previsões conservadoras
    if true_action in valueable_categorias and action in valueable_categorias:
      reward += extra_good_reward
    return reward
  else:
    penalty = bad_error_penalty
    if true_action in valueable_categorias and action in valueable_categorias:
      penalty += extra_bad_penalty
    return penalty


def normalize_data(features):
    scaler = StandardScaler()
    return scaler.fit_transform(features)

def main():
    features = data[['odd_entrada',
       'odd_categoria_media_movel5', 'odd_categoria_media_movel10',
       'odd_categoria_media_movel20', 'odd_categoria_media_movel40',
       'odd_categoria_media_movel80', 'odd_categoria_media_movel160',
       'odd_categoria_media_movel320', 'odd_categoria_media_movel640',
       'odd_mediana20', 'odd_min10', 'odd_min20', 'odd_min40', 'odd_max10',
       'tendencia_media_movel10', 'categoria_frequencia20',
       'entropia_rolling5', 'entropia_rolling320',
       'correlacao_odd_categoria_media_movel10',
       'correlacao_odd_categoria_media_movel640', 'categoria_contagem',
       'variancia_rolling10', 'variancia_rolling20', 'variancia_rolling320',
       '1entrada-anterior', '2entrada-anterior', '3entrada-anterior',
       '4entrada-anterior', '5entrada-anterior', '6entrada-anterior',
       '7entrada-anterior', '8entrada-anterior', '9entrada-anterior',
       '10entrada-anterior']].values
    rewards = data['Odd_Categoria'].values

    features = normalize_data(features)
    X_train, X_test, y_train, y_test = train_test_split(features, rewards, test_size=0.2, random_state=42)

    n_epochs = 10
    batch_size = 32
    partition_size = 5000  # Tamanho de cada partição

    n_states = X_train.shape[1]
    n_actions = len(np.unique(y_train))
    dqn_agent = DQNAgent(n_states, n_actions, learning_rate=0.001, gamma=0.99, epsilon=1.0, buffer_size=5000)

    for epoch in range(n_epochs):
        for partition_start in range(0, len(X_train), partition_size):
            partition_end = min(partition_start + partition_size, len(X_train))
            X_train_partition = X_train[partition_start:partition_end]
            y_train_partition = y_train[partition_start:partition_end]

            for i in range(len(X_train_partition) - 1):
                state = X_train_partition[i, :]
                action = dqn_agent.get_action(state)
                true_action = y_train_partition[i]
                reward = calculate_reward(action, true_action)
                next_state = X_train_partition[i + 1, :] if i + 1 < len(X_train_partition) else X_train_partition[0, :]
                done = (i == len(X_train_partition) - 2)

                dqn_agent.remember(state, action, reward, next_state, done)

                if done:
                    dqn_agent.replay(batch_size)
                    dqn_agent.update_epsilon(0.995, 0.01)


    # Teste do agente
    test_accuracy = []
    test_directional_accuracy = []
    test_weighted_directional_accuracy = []

    for partition_start in range(0, len(X_test), partition_size):
        partition_end = min(partition_start + partition_size, len(X_test))
        X_test_partition = X_test[partition_start:partition_end]
        y_test_partition = y_test[partition_start:partition_end]

        correct_predictions = 0
        overestimations = 0
        weighted_correct_predictions = 0
        weighted_overestimations = 0

        for i in range(len(X_test_partition)):
            state = X_test_partition[i, :]
            action = dqn_agent.get_action(state)
            true_action = y_test_partition[i]

            if action == true_action:
                correct_predictions += 1
                weighted_correct_predictions += 2
            elif action > true_action:
                overestimations += 1
                weighted_overestimations += 1

        accuracy = correct_predictions / len(X_test_partition)
        directional_accuracy = (correct_predictions + overestimations) / len(X_test_partition)
        weighted_directional_accuracy = (weighted_correct_predictions + weighted_overestimations) / len(X_test_partition)

        test_accuracy.append(accuracy)
        test_directional_accuracy.append(directional_accuracy)
        test_weighted_directional_accuracy.append(weighted_directional_accuracy)

    # Calcular média das métricas de teste
    avg_accuracy = np.mean(test_accuracy)
    avg_directional_accuracy = np.mean(test_directional_accuracy)
    avg_weighted_directional_accuracy = np.mean(test_weighted_directional_accuracy)

    print("Precisão Média no Teste:", avg_accuracy)
    print("Acurácia Direcional Média no Teste:", avg_directional_accuracy)
    print("Acurácia Direcional Ponderada Média no Teste:", avg_weighted_directional_accuracy)

    dqn_agent.save_model('Agente5000.h5')

if __name__ == "__main__":
    main()

A saída de streaming foi truncada nas últimas 5000 linhas.
1/1 [==============================] - 1s 1s/step
